<a href="https://colab.research.google.com/github/vimigueloli/ICPD/blob/main/interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ICPD inteligencia classificadora para derrame


sexo='masculino' #@param ['masculino','feminino']
idade= 0 #@param {type:"slider", min:0, max:115} 
hipertensao= False #@param {type:"boolean"}
doenca_do_coracao= False #@param {type:"boolean"}
ja_casou= False #@param {type:"boolean"}
tipo_de_trabalho= 'autonomo'#@param ['autonomo', 'privado', 'governamental', 'criança', 'nunca trabalhou']
vive_na_cidade= True #@param {type:"boolean"}
nivel_de_glucose= 80 #@param {type:"slider", min:50, max:170} 
peso= 75 #@param {type:"slider", min:1, max:200, step: 0.1} 
altura= 0.5 #@param {type:"slider", min:0.15, max:2.30, step: 0.01}
fumante='formalmente' #@param ['formalmente','nunca fumou','fumante']